Nello script "OriginalDataProject" è stato scelto il miglior algortimo di imputazione. A questo punto estraggo dal dataset principale estreggo circa 800 items per cui ho la classificazione in "INTESTRAT-CAD_TC_class.xlsx". Sarà su questo database che effettuero l'imputing di tutte le colonne.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import statsmodels as stm
import missingno as msno
import statsmodels.imputation.mice as mice
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import random 
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
import os 

#Kepler Mayer
import sklearn
from sklearn import ensemble
import kmapper as km
from sklearn import preprocessing
from kmapper.plotlyviz import *
from sklearn.manifold import TSNE
import umap.umap_ as umap
import warnings
warnings.filterwarnings("ignore")
import plotly.graph_objs as go
from ipywidgets import (HBox, VBox)
import sys
import operator
from collections import Counter
from collections import OrderedDict
from sklearn.metrics.pairwise import cosine_similarity
import gower as gw 

#Per prima cosa verifichiamo la normalità dei dati
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot
import math
import pathlib
import scipy.stats as scy

# To use the IterativeImputer, we need to explicitly ask for it:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor

import plotly.graph_objs as go
from ipywidgets import (HBox, VBox)
import plotly.graph_objects as go
import plotly.io as pio

#Funzioni per la NetworkX 
from networkx.algorithms import community
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
import networkx.algorithms.community as nxcom


from pySankey.sankey import sankey

rnd = np.random.RandomState(17489)
st = random.getstate()  # remeber this state 

# Definisco una color-map che associa un colore ai diversi valori in base al bin di appartenenza
pl_brewer = [[0.0, '#fcefb4'],  
             [0.1, '#FFBA08'],
             [0.2, '#FAA307'],
             [0.3, '#F48C06'],
             [0.4, '#E85D04'],
             [0.5, '#DC2F02'],
             [0.6, '#D00000'],
             [0.7, '#9D0208'],
             [0.8, '#6A040F'],
             [0.9, '#370617'],
             [1.0, '#03071E']]

In [ ]:
#Funzioni
def read_csv(path):
    """
    Funzione necessaria a leggere un file csv senza che "dtype" delle diverse variabili
    venga perso.
    INPUT:
    - path: percorso e nome del file da salvare
    OUTPUT:
    salvataggio del file
    """
    # Leggo dalla prima riga il tipo delle rispettive variabili
    dtypes = pd.read_csv(path, nrows=1).iloc[0].to_dict()
    # Leggo il resto delle righe e quindi i dati
    return pd.read_csv(path, dtype=dtypes, skiprows=[1])


def to_csv(df, path):
    """
    Funzione necessaria per poter conservare il "dtype" anche nella fase di salvataggio e 
    successiva rilettura.
    INPUT:
    - df: dataframe da salvare
    - path: percorso e nome del file da salvare
    """
    # Salvo il tipo delle variabili salvandolo come nuova riga nel file
    df.loc[-1] = df.dtypes
    df.index = df.index + 1
    df.sort_index(inplace=True)
    # Successivamente salvo il file csv
    df.to_csv(path, index=False)
    
def find_file(path,df,flag):
    """
    Funzione che permette di verificare se un file è presente o meno
    INPUT:
    - path: percorso del file con nome finale del file
    - df: dataframe che si vuole salvare
    - flag: segnale che mi permette di salvare e quindi sovrascrivere il file anche se presente
    """
    if not(flag):
        if os.path.isfile(path):
            print("Il file esiste! Se vuoi salvare comunque il file cambia il flag a 1")
        else:
            print("Il file non esiste! Verrà salvato nella cartella File Memory")
            to_csv(df,path)
            
    else:
        to_csv(df,path)


def entropy_count(scomplex):
    """
    Funzione che mi permette di calcolare l'entropia del grafo.
    INPUT:
    - scomplex: complesso simpliciale
    OUTPUT:
    - entropy_graph: entropia media del grafo
    """
    kmgraph = get_mapper_graph(scomplex)
    # assign to node['custom_tooltips']  the node label (0 for absence of  cardiac insufficiency, 1 for different type of cardiac insufficiency)
    for node in kmgraph[0]['nodes']:
        node['custom_tooltips'] = y_class[scomplex['nodes'][node['name']]]
        Entropy_node ={}
    for j, node in enumerate(kmgraph[0]['nodes']):   #j è l'id del nodo mentre in node vi sono le caratteristiche del nodo
        dimCluster = node['cluster']['size']
        data = node['custom_tooltips']
        dimData0 = len(data.loc[data==0])
        dimData1 = len(data.loc[data==1])
        if dimData0 != 0 and dimData1 !=0:
            H_node = -((dimData0/dimCluster)*math.log2(dimData0/dimCluster)+(dimData1/dimCluster)*math.log2(dimData1/dimCluster))
        if dimData0 != 0 and dimData1 ==0:
            H_node = -((dimData0/dimCluster)*math.log2(dimData0/dimCluster))
        if dimData0 == 0 and dimData1 !=0:
            H_node = -((dimData1/dimCluster)*math.log2(dimData1/dimCluster))
                
        Entropy_node[j] = [dimCluster, H_node]
    sumEntropies = 0
    numberData = 0
    for j, node in enumerate(Entropy_node):
        sumEntropies = sumEntropies + (Entropy_node[j][0]*Entropy_node[j][1])
        numberData = numberData + Entropy_node[j][0]
    entropy_graph = sumEntropies/numberData
    return entropy_graph



def do_graph(scomplex, title, y_class, my_colorscale):
    """
    Funzione che genera il grafico tramite plotly
    INPUT:
    - scomplex: complesso simpliciale
    - title: titolo del grafico
    - y_class: vettore contenente la classificazione 
    my_colorscale: color map utilizzata per colorare il grafo
    OUTPUT:
    - fwn: figure widget contenente il grafico
    """
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex,
                                                                 color_function_name='Distance to x-min',
                                                                 colorscale=my_colorscale)
    
    
    # assegno al node['custom_tooltips'] il label (0 per CAD = 0, 1 per CAD = 1,2)
    for node in kmgraph['nodes']:
        node['custom_tooltips'] = y_class[scomplex['nodes'][node['name']]]
    
    bgcolor = '#e9ecef'    #è il colore dello sfondo su cui vi è la rete
    y_gridcolor = 'rgb(150,150,150)'# è il colore della griglia

    plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale,
                                 factor_size=4.0, edge_linewidth=1.0)
    layout = plot_layout(title='Topological network representing the<br>  cardiac insufficiency',
                     width=700, height=750,
                     annotation_text=title + "<br>" + get_kmgraph_meta(mapper_summary), 
                     bgcolor=bgcolor)

    fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)


    heartFail_dict = {0: 'Class_0', 1: 'Class_1'}
    tooltips = list(fw_graph.data[1].text) # eseguo questo cast perchè fw.data[1].text è una tupla che vogliamo aggiungere al
                                     # tooltips

    new_color = []  #è un vettore tanto lungo quanti sono i nodi e con numeri tra 0 e 1
    for j, node in enumerate(kmgraph['nodes']):
        member_label_ids = y_class[scomplex['nodes'][node['name']]]         #al node j-esimo associo 0-1 in base alla classe  (se volessi colorare rispetto ad altro basta modificare questa riga e il rispettivo dict)
        member_labels = [heartFail_dict[id] for id in member_label_ids]     #in questo modo ad opgni nodo viene creato un vettore contentente class0 o class1 in base ai pazienti che apprtengono a quel nodo
        label_type, label_counts = np.unique(member_labels, return_counts=True)  #in questo modo so la frequenza della classi nel nodo e in ogni nodo

        n_members = label_counts.sum()
        if label_type.shape[0] == 1:          # Se presente solo la classe 1 o classe 0 vuol dire che il vettore ha solo una dimensione 
            if label_type[0] == 'Class_0':
                new_color.append(0.0)           # Associo il colore basso del range
            else:
                new_color.append(1.0)           # Associo il colore alto del range
        else:
            new_color.append(1.0*label_counts[1]/n_members)   # Associo il colore in base alla proporzione della Classe 0

        for m in range(len(label_counts)):    # Associo ad ogni nodo "cluster" il numero di elementi 
            tooltips[j] += '<br>' + str(label_type[m]) + ': ' + str(label_counts[m]) # append  how many different

    
    
    fwn_graph = go.FigureWidget(fw_graph) # recupero la figura creata precedentemente
    with fwn_graph.batch_update(): # eseguo l'update dalla nuova figure
        fwn_graph.data[1].text = tooltips # aggiungo i nuovi tooltips
        fwn_graph.data[1].marker.colorbar.x = -0.14 # definisco la posizione della toolbar
        fwn_graph.layout.width = 550 # definisco le dimensioni della finestra per ottenere due copie parallele
        fwn_graph.layout.height = 550
        fwn_graph.layout.margin.r = 20 # diminuisco il margine destro da 60px (default val) a 45 pixels
        fwn_graph.data[1].marker.color = new_color # aggiorno i colori dei nodi

    return fwn_graph

        
def do_enrinchment(scomplex,my_colorscale, var_cont, var_cont_norm, fwn,  title):
    """
    Funzione che permette di colorare la rete ricavata dalla TDA rispetto ad una determinata
    variabile (continua) che caratterizza i pazienti appartenenti ai nodi
    INPUT:
    - scomplex: complesso simpliciale ottenuto dal package TDA Mapper
    - my_colorscale: color map scelta
    - var_cont: variabile continua
    - var-cont_norm: variabile continua normalizzata
    - fwn: figure widget contenente la rete colorata rispetto alla classe CAD
    - title: titolo da inserire nel grafico
    OUTPUT:
    - fwn: figure widget contenente il grafico
    """
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex,
                                                                 color_function_name='Distance to x-min',
                                                                 colorscale=my_colorscale)
    
    tooltipsVar = list(fwn.data[1].text) # eseguo questo cast perchè fw.data[1].text è una tupla che vogliamo aggiungere al
                                     # tooltips       
    heartFail_dict = {0: 'Class_0', 1: 'Class_1'}
    new_color_var = []  #è un vettore tanto lungo quanti sono i nodi e con numeri tra 0 e 1
    for j, node in enumerate(kmgraph['nodes']):
        member_label_ids = y_class[scomplex['nodes'][node['name']]], var_cont[scomplex['nodes'][node['name']]], var_cont_norm[scomplex['nodes'][node['name']]]          #al node j-esimo associo 0-1 in base alla classe e all'età (se volessi colorare rispetto ad altro basta modificare questa riga e il rispettivo dict)
        member_labels = [heartFail_dict[id] for id in member_label_ids[0]]     #in questo modo ad ogni nodo viene creato un vettore contentente class0 o class1 in base ai pazienti che appartengono a quel nodo
        label_type, label_counts = np.unique(member_labels, return_counts=True)  #in questo modo so la frequenza della classi nel nodo e in ogni nodo
        mean_var_members = member_label_ids[1].mean()
        n_members = label_counts.sum()
    
        #Associo il colore direttamente in base all'età media che essendo scalata avrà valore tra 0(media bassa) e 1
        new_color_var.append(member_label_ids[2].mean())
        tooltipsVar[j] += '<br>' + "Mean:" + str(mean_var_members)  #In questo modo visualizzero l'età media del nodo (non rispetto alla classe)
           
    fwVar = go.FigureWidget(fwn) #definisco la nuova Figure da fwn_graph che verrà colorato in modo diverso
    with fwVar.batch_update():
        fwVar.data[1].marker.color = new_color_var # carico i nuovi colori
        fwVar.data[0].line.color = 'rgb(125,125,125)' # carico il nuovo colore degli edges
        fwVar.data[1].text = tooltipsVar # aggiungo la nuova tooltips
        fwVar.layout.plot_bgcolor = '#edf2ef'
        fwVar.layout.annotations = None # rimuovo il mapper_summary dal secondo plot
        fwVar.data[1].marker.showscale = False # rimuovo la color bar
        fwVar.layout.title = "Nodes are colored according to the average<br>"+ "members's" + title 
    return fwVar


def do_enrinchment_cat(scomplex,my_colorscale, var_cat, fwn,  title, cat_dict):
    """
    Funzione che permette di colorare la rete rispetto ad una variabile categorica a due livelli
    che caratterizza i pazienti appartenenti ai singoli nodi.
    INPUT:
    - scomplex: complesso simpliciale ottenuto dal package TDA Mapper
    - my_colorscale: color map scelta
    - var_cat: variabile continua
    - title: titolo del grafico 
    - fwn: figure widget contenente la rete colorata rispetto alla classe CAD
    - cat_dict: dizionario contenente il label della variabile categorica
    OUTPUT:
    - fwn: figure widget contenente il grafico
    """
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex,
                                                                 color_function_name='Distance to x-min',
                                                                 colorscale=my_colorscale)
    
    tooltipsVar = list(fwn.data[1].text)# eseguo questo cast perchè fw.data[1].text è una tupla che vogliamo aggiungere al
                                     # tooltips
    new_color = []  #è un vettore tanto lungo quanti sono i nodi e con numeri tra 0 e 1
    for j, node in enumerate(kmgraph['nodes']):
        member_label_ids = y_class[scomplex['nodes'][node['name']]], var_cat[scomplex['nodes'][node['name']]]          #al node j-esimo associo 0-1 in base alla classe e all'età (se volessi colorare rispetto ad altro basta modificare questa riga e il rispettivo dict)
        member_labels = [cat_dict[id] for id in member_label_ids[1]]     #in questo modo ad ogni nodo viene creato un vettore contentente class0 o class1 in base ai pazienti che appartengono a quel nodo
        label_type, label_counts = np.unique(member_labels, return_counts=True)  #in questo modo so la frequenza della classi nel nodo e in ogni nodo
        
        n_members = label_counts.sum()
        if label_type.shape[0] == 1:          # Se presente solo la classe 1 o classe 0 vuol dire che il vettore ha solo una dimensione 
            if label_type[0] == 'Cat_0':
                new_color.append(0.0)           # Associo il colore basso del range
            else:
                new_color.append(1.0)           # Associo il colore alto del range
        else:
            new_color.append(1.0*label_counts[1]/n_members)   # Associo il colore in base alla proporzione della Classe 0

        for m in range(len(label_counts)):    # Associo ad ogni nodo "cluster" il numero di elementi 
            tooltipsVar[j] += '<br>' + str(label_type[m]) + ': ' + str(label_counts[m]) # appendo le quantità dei diversi label    
    
    fwVar = go.FigureWidget(fwn) #definisco la nuova Figure da fwn_graph che verrà colorato in modo diverso
    with fwVar.batch_update():
        fwVar.data[1].marker.color = new_color # aggiorno il colore dei nodi
        fwVar.data[0].line.color = 'rgb(125,125,125)' # aggiorno il colore degli edges
        fwVar.data[1].text = tooltipsVar # aggiungo i nuovi tooltips
        fwVar.layout.plot_bgcolor = '#edf2ef'
        fwVar.layout.annotations = None # rimuovo il mapper_summary dal secondo plot
        fwVar.data[1].marker.showscale = False #  rimuovo la colorbar
        fwVar.layout.title = "Nodes are colored according to the average<br>"+ "members's" + title 
    return fwVar


def do_hist_counter(scomplex,title,path):
    """
    Funzione che permette di calcolare la distribuzione della numerosità dei diversi nodi che
    caratterizzano la rete.
    INPUT:
    - scomplex: complesso simpliciale ottenuto dal package TDA Mapper
    - title: titolo del grafico 
    - path: percorso e nome del file da salvare
    OUTPUT:
    - bar_dist: 
    """    
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex,
                                                                 color_function_name='Distance to x-min')
    
    counter = list()
    for j, node in enumerate(kmgraph['nodes']):
        counter.append(node["cluster"]["size"])
    counter_dist = Counter(counter)
    counter_dist = OrderedDict(sorted(counter_dist.items(), key=lambda kv: kv[0]))
    
    #Creo l'immagine 
    chiavi = list(counter_dist.keys())
    valori = list(counter_dist.values())
    
    ax  = plt.figure(figsize=(10,7))  
    barDist = sns.barplot(x=chiavi, y= valori, palette="Blues_d")
    plt.suptitle(title)
    plt.savefig(path,width = 300, height =300)

    return barDist


#Funzioni per la gestione del NetworkX
def set_node_community(G, communities):
        """
        Funzione che aggiunge alla community  il nodo come attributo.
        INPUT:
        - G: grafo    
        - communities: comminities trovate
        """
        for c, v_c in enumerate(communities):
            for v in v_c:
                # Aggiungo 1 per salvare l'edge esterno
                G.nodes[v]['community'] = c + 1

def set_edge_community(G):
        """
        Funzione che cerca gli edges interni alla communities e li aggiunge.
        INPUT:
        - G: grafo        
        """
        for v, w, in G.edges:
            if G.nodes[v]['community'] == G.nodes[w]['community']:
                # I lati interni vengono marcati nella community
                G.edges[v, w]['community'] = G.nodes[v]['community']
            else:
                # I lati esterni vengono marcati con 0
                G.edges[v, w]['community'] = 0

def get_color(i, r_off=1, g_off=1, b_off=1):
        """
        Funzione che assegna un colore ai vertici della stessa communities.
        INPUT:
        i: contatore 
        """
        
        r0, g0, b0 = 0, 0, 0
        n = 16
        low, high = 0.1, 0.9
        span = high - low
        r = low + span * (((i + r_off) * 3) % n) / (n - 1)
        g = low + span * (((i + g_off) * 5) % n) / (n - 1)
        b = low + span * (((i + b_off) * 7) % n) / (n - 1)
        return (r, g, b)
    
def do_color(scomplex, var_cont, var_cont_norm, dict_class,my_colorscale):
    """
    Funzione che permette di colorare la rete rispetto ad una variabile continua
    INPUT:
    - scomplex: complesso simpliciale ottenuto dal package TDA Mapper
    - var_cont: variabile continua
    - var_cont_norm: variabile continua normalizzata
    - dict_class: dizionario contenente i labels della classe
    - my_colorscale: color map scelta
    OUTPUT:
    - new_color_var: vettore contenete il colore di ogni nodo 
    """
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex,
                                                                 color_function_name='Distance to x-min',
                                                                 colorscale=my_colorscale)

    new_color_var= []  #è un vettore tanto lungo quanti sono i nodi e con numeri tra 0 e 1
    for j, node in enumerate(kmgraph['nodes']):
        member_label_ids = y_class[scomplex['nodes'][node['name']]], var_cont[scomplex['nodes'][node['name']]], var_cont_norm[scomplex['nodes'][node['name']]]          #al node j-esimo associo 0-1 in base alla classe e all'età (se volessi colorare rispetto ad altro basta modificare questa riga e il rispettivo dict)
        member_labels = [dict_class[id] for id in member_label_ids[0]]     #in questo modo ad ogni nodo viene creato un vettore contentente class0 o class1 in base ai pazienti che appartengono a quel nodo
        label_type, label_counts = np.unique(member_labels, return_counts=True)  #in questo modo so la frequenza della classi nel nodo e in ogni nodo
        mean_var_members = member_label_ids[1].mean()
        n_members = label_counts.sum()
    
        #Associo il colore direttamente in base all'età media che essendo scalata avrà valore tra 0(media bassa) e 1
        new_color_var.append(member_label_ids[2].mean())

    return new_color_var 

def do_color_class(scomplex,y_class,dict_class,my_colorscale):
    """
    Funzione che permette di colorare la rete rispetto alla classe CAD
    INPUT:
    - scomplex: complesso simpliciale ottenuto dal package TDA Mapper
    - y_class: vettore classificazione
    - dict_class: dizionario contenente i labels della classe
    - my_colorscale: color map scelta
    OUTPUT:
    - new_color_class: vettore contenete il colore di ogni nodo 
    """
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex,
                                                                 color_function_name='Distance to x-min',
                                                                 colorscale=my_colorscale)
    
    new_color_class = []  #è un vettore tanto lungo quanti sono i nodi e con numeri tra 0 e 1
    for j, node in enumerate(kmgraph['nodes']):
        member_label_ids = y_class[scomplex['nodes'][node['name']]]         #al node j-esimo associo 0-1 in base alla classe  (se volessi colorare rispetto ad altro basta modificare questa riga e il rispettivo dict)
        member_labels = [dict_class[id] for id in member_label_ids]     #in questo modo ad opgni nodo viene creato un vettore contentente class0 o class1 in base ai pazienti che apprtengono a quel nodo
        label_type, label_counts = np.unique(member_labels, return_counts=True)  #in questo modo so la frequenza della classi nel nodo e in ogni nodo

        n_members = label_counts.sum()
        if label_type.shape[0] == 1:          # Se presente solo la classe 1 o classe 0 vuol dire che il vettore ha solo una dimensione 
            if label_type[0] == 'Class_0':
                new_color_class.append(0.0)           # Associo il colore basso del range
            else:
                new_color_class.append(1.0)           # Associo il colore alto del range
        else:
            new_color_class.append(1.0*label_counts[1]/n_members)   # Associo il colore in base alla proporzione della Classe 0
    return new_color_class

#Con questa funzione va a cercare le community all'interno della rete
def community_searching(G):
    """
    Funzione che permette di applicare l'algoritmo di ricerca 
    delle community all'interno della rete
    INPUT:
    - G: grafo
    OUTPUT:
    - node_color: colore dei nodi 
    - internal: lista dei nodi appartenenti alle communities
    - internal_color: colore delle diverse communities
    - external: nodi esterni
    """
    #Estraggo le communieties all'interno del grafo tramite metodo 'Greedy_modularity'
    communities =girvan_newman(G)
    # Cerco le communities a partire dalla più grande
    communities = sorted(nxcom.greedy_modularity_communities(G), key=len , reverse=True)

    # Definisco i nodi e gli edges della communities
    set_node_community(G, communities)
    set_edge_community(G)

    node_color = [get_color(G.nodes[v]['community']) for v in G.nodes]

    # Importo il colore degli edges tra membri della stessa communities (interni)
    # e degli edges tra le diverse communities
    external = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] == 0]
    internal = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] > 0]
    internal_color = ['black' for e in internal]
    return node_color, internal, internal_color,external



def do_graph_NetworkX(G,color_class, path, node_color, internal, internal_color):
    """
    Funzione che permette di ricavare due grafici a confronto partendo dal complesso simpliciale
    INPUT:
    - G: grafo 
    - color_class: vettore contenente il colore dei nodi rispetto alla classe
    - path: percorso e nome del file in cui si vuole salvare l'immagine 
    - node_color: vettore contenente il colore dei nodi rispetto alle communities
    - internal: lista dei lati delle commmunities interne
    - internal_color: colore dei nodi appartenenti alle diverse communities
    """
    plt.rcParams.update({'figure.figsize': (15, 10)})
    # Grafico la stessa rete coni pacchetti NetworkX e igraph
    fig, (ax0,ax1) = plt.subplots(nrows=1, ncols=2, figsize=(14, 8))

    ax0.set_title("Plot with NetworkX - Class Proportion")
    nx.colormaps = pl_brewer
    nx.draw_kamada_kawai(G, node_size=80, ax=ax0, node_color=color_class, cmap = 'coolwarm')
    #la color map coolwarm va dal blu (0) al rosso (1) quindi nei nodi blu vi è una prevalenza di soggetti sani

    ax1.set_title("Plot with NetworkX - Communities")
    nx.draw_kamada_kawai(G, node_size=50, ax=ax1, node_color= node_color, edgelist=internal, edge_color = internal_color)

    plt.savefig(path)


In [ ]:
#Carico il database "INTESTRAT-CAD_TC_class.xlsx" 
db_class = pd.read_excel("C:/Users/Antonio Montanaro/Desktop/Tesi Magistrale/File Memory/INTESTRAT-CAD_TC_class.xlsx")

Le righe del db_class sono 829 ciò vuol dire che mi aspetto ci siano 829 pazienti

In questa database troviamo la classificazione completa, classificazione molto utile poichè è assente per qualche paziente nel database generale. 

In [ ]:
#Elimino la riga in cui è contenuto il paziente con "STENT" poichè non ha nessuna classificazione 
db_class.drop(db_class.loc[db_class["Class"]=='STENT'].index, inplace = True)

#Elimino le righe dei pazienti in cui non si ha una classificazione
#Salvo l'ID dei pazienti per cui non ho classificazione
ID_noClass = db_class.loc[db_class["Class"].isna()].index   
db_class.drop(db_class.loc[db_class["Class"].isna()].index, inplace= True)

Dopo aver cancellato la riga del paziente con Class = STENT e quelle dei pazienti senza classificazione, i pazienti diventano 826

In [ ]:
#Classificazione
print("Pazienti con Classe 0 = ",len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 0.0]))
print("Pazienti con Classe 1 = ",len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 1.0]))
print("Pazienti con Classe 2 = ",len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 2.0]))
print("Pazienti Totali = ",len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 0.0]) + len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 1.0]) + len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 2.0]))

In [ ]:
dataSet = read_csv("C:/Users/Antonio Montanaro/Desktop/Tesi Magistrale/File Memory/dataset_Visita1.csv")
del dataSet["Event Name"]
del dataSet["Data della visita"]
del dataSet["Classe NYHA"]
del dataSet["Se sì, specificare la Class Canadian Society"]

A questo punto vado ad estrapolare dal dataset principale "dataset" tutti gli items per cui vi è il matching con il codice epifania contenuto in db_class. In questo modo vado ad ottenere un dataset composto dai soli items che hanno una classificazione.

In [ ]:
print("Considerando la Classe CAD 0-1-2:")
print("Soggetti con Classe 0:",len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 0.0]))
print("Soggetti con Classe 1:",len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 1.0]))
print("Soggetti con Classe 2:",len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 2.0]))
print("\n")
print("Considerando la Classe CAD 0 vs 1-2:")
print("Soggetti con Classe 0:",len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 0.0]))
print("Soggetti con Classe 1 e 2:",len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 1.0]) + len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 2.0]))

fig = plt.figure(figsize=(5, 5))
ax = fig.add_axes([0,0,1,1])
class_labels = ['CAD 0', 'CAD 1', 'CAD 2']
values = [len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 0.0]), len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 1.0]),len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 2.0])]
ax.bar(class_labels,values, color = ["#fe5f55", "#ffba08","#648de5"])
plt.title("Classificazione CAD 0 vs 1 vs 2")
plt.show()

fig = plt.figure(figsize=(5, 5))
ax = fig.add_axes([0,0,1,1])
class_labels = ['CAD 0', 'CAD 1-2']
values = [len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 0.0]), len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 1.0])+len(db_class.loc[db_class["CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)"] == 2.0])]
ax.bar(class_labels,values, color = ["#fe5f55","#648de5"])
plt.title("Classificazione CAD 0 vs 1-2")
plt.show()

In [ ]:
dataSet.rename(columns = {'Codice Epifania':'Sample_ID'}, inplace = True)
dataSet[dataSet["Sample_ID"].isin(db_class["Sample_ID"].tolist())]
dataSet1 = dataSet.copy()
db = dataSet1.merge(db_class)

In [ ]:
print("lunghezza del dataset mergiato:" + str(len(db)))
print("lunghezza del dataset con classificazione:" + str(len(db_class)))
#Tra uno e l'altro mancono 3 pazienti, perchè? 

In [ ]:
SampleNotInClass = set(db_class["Sample_ID"].tolist()) - set(dataSet["Sample_ID"].tolist())
SampleInClass = set(dataSet["Sample_ID"].tolist())
np.save("SampleWithoutClass",list(SampleNotInClass))
np.save("SampleClass",list(SampleInClass))

In SampleNotInClass sono presenti quei codici epifania che sono presenti nella dataSet completto ma per cui non vi è una classificazione in db_class. Questo spiega perchè la lunghezza finale del database contentente i pazienti sia con una classificazione in db_class che i dati in dataSet è di 823.

In [ ]:
#Rinomino la colonna CAD poichè è quella contenente la classificazione e in questo modo sono 
#anche sicuro che non perderò l'allineamento riga del dataset con riga del vettore con la classificazione
dataSet = db
dataSet.rename(columns = {'CAD (0: stenosi 0%; 1: stenosi 1-69%; 2: stenosi 70-100% o significativa)':'Class1'}, inplace = True)
y_class = dataSet["Class1"].apply(lambda x: 0 if x == 0.0 else 1)
y_class2 = dataSet["Class1"]

find_file("../File Memory/datasetFinale_NonImputato.csv",dataSet,0)

dataSet = read_csv("C:/Users/Antonio Montanaro/Desktop/Tesi Magistrale/File Memory/datasetFinale_NonImputato.csv")
del dataSet["Class1"]
del dataSet["TC clinica"]
del dataSet["Equi"]

Ottengo i due dataset "db" e "dataset", in particolare nel primo sono anche contenute tutte le colonne di entrambi i dataset mentre il secondo contiene solo gli items (quindi i pazienti) il cui codice epifinia è contenuto nel database della classe. A questo punto faccio l'imputing del database sfruttando il Miss Forest, stando però attento a:
1) Normalizzare il dataset;
2) Distringuere la variabili continue da quelle categoriche

In [ ]:
id_paz = dataSet["ID PAZIENTE"]
del dataSet["ID PAZIENTE"]
del dataSet["Sample_ID"]
imputer = IterativeImputer(missing_values = np.nan, estimator = ExtraTreesRegressor(n_estimators=10, random_state=0))
Ximp = imputer.fit_transform(dataSet)

In [ ]:
find_file("../File Memory/id_paz.csv", id_paz, 0)

In [ ]:
#Per ora lo separo come cella in modo da non dover runnare ogni volta l'Isoforest
datasetImputed = pd.DataFrame(Ximp, columns = dataSet.columns)

Dato che dopo l'imputing perdo l'informazione del tipo delle singolo colonne, posso ricavare tale informazione dal database non imputanto visto che i due dataset (imputato e non) sono indentici dal punto di vista della struttura.

In [ ]:
cat_features = dataSet.columns[dataSet.dtypes == 'category']
datasetImputed[cat_features] = datasetImputed[cat_features].astype("int") #Converto prima le colonne che saranno categoriche in intero
datasetImputed[cat_features] = datasetImputed[cat_features].astype(dtype = "category") #Dopodiche associo il tipo

#Estraggo dal dataset le variabili continue
cont_features = dataSet.columns[dataSet.dtypes != 'category']

#Estraggo dal dataset le variabili categoriche
find_file("../File Memory/datasetFinale_Imputato.csv",datasetImputed.copy(),0)


Dopo aver ottenuto il dataset imputato lo salvo in modo da poterlo conservare per utilizzi successivi senza dover ogni volta rieseguire l'intero algoritmo. Successivamente valuto la bontà dell'imputing tramite un confronto tra db imputato e db non imputato

In [ ]:
#Mi salvo una copia del dataset senza dummy
dataNDumm = datasetImputed
#Trasformo le variabili categoriche in dummy, in modo tale che dopo posso effettuare lo scaling
XcatFeatures =  datasetImputed[cat_features]
XcatND = XcatFeatures #servirà una copia senza dummy variable

XcatFeatures = pd.get_dummies(XcatFeatures, columns=cat_features.delete(0), drop_first=True)
XcatFeatures["ID PAZIENTE"] = id_paz #aggiungo questa colonna in modo poi da poter rifare il merge con il dataframe continuo

#Isolo il dataframe continuo
XcontFeatures = datasetImputed[cont_features]
XcontFeatures["ID PAZIENTE"] = id_paz

#Faccio il merge tra i due dataset scomposti
datasetImputed = pd.merge(left=XcontFeatures, right=XcatFeatures, how='outer', on='ID PAZIENTE')
del datasetImputed["ID PAZIENTE"]

#Recupero il nome delle categoriche e continue
cat_features = datasetImputed.columns[datasetImputed.dtypes == 'uint8']


In [ ]:
#Per prima cosa vado ad eseguire lo scaling del mio dataset
dataImputed = datasetImputed
X = datasetImputed.values #trasformo il dataframe in una matrix
min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)
datasetImputed = pd.DataFrame(X_scaled, columns = datasetImputed.columns)
#Ristabilisco il tipo delle colonne
datasetImputed[cat_features] = datasetImputed[cat_features].astype("int") #Converto prima le colonne che saranno categoriche in intero
datasetImputed[cat_features] = datasetImputed[cat_features].astype(dtype = "category") #Dopodiche associo il tipo

Dopo aver ottenuto il dataset completo e pronto per l'analisi, eseguo la parte di algoritmo che mi permette di poter valutare le lenti che sono stasticamente significative nel fare il washout della classe

In [ ]:
# Poichè il dataSet X è eterogeneo affinchè il TDA funzioni in modo adeguato è necessario che X venga reso omogeneo 
# attraverso una misura di similarità, in questo caso similarità coseno e gower
X1 = cosine_similarity(X_scaled,X_scaled)
X2 = gw.gower_matrix(X_scaled)

#Salvo le matrici delle distanze coseno e gower in formato csv 
np.savetxt("GowerMatrix.csv", X2, delimiter=",")
np.savetxt("CosineMatrix.csv", X1, delimiter=",")
np.savetxt("Class.csv", y_class, delimiter=',')

In [ ]:
feature_names = dataSet.columns
feature_names = feature_names.delete(len(feature_names)-1)

mapper = km.KeplerMapper(verbose=2)

# We create a custom 1-D lens with Isolation Forest
model = ensemble.IsolationForest(random_state=rnd)
lens1 = model.fit(X2)
lens1 = model.decision_function(X2).reshape((X2.shape[0], 1))
IsoForest_lens = pd.Series(np.concatenate(lens1))  #Cast into Pandas Series

# We create another 1-D lens with L2-norm
lens2 = mapper.fit_transform(X2, projection="l2norm", distance_matrix = None, scaler = None)
L2Norm_lens = pd.Series(np.concatenate(lens2))  #Cast into Pandas Series

# We create another 1-D lens with one component of PCA
lens3 = mapper.fit_transform(X2, projection=sklearn.decomposition.PCA(n_components=1, random_state=rnd), distance_matrix = None, scaler = None)
PCA_lens = pd.Series(np.concatenate(lens3))  #Cast into Pandas Series


# We create another 1-D lens with with one component of UMAP
lens4 = mapper.project(X2, projection=umap.UMAP(n_components=1, random_state=rnd), distance_matrix = None, scaler = None)
Umap1_lens = pd.Series(np.concatenate(lens4))  #Cast into Pandas Series

# We create another 1-D lens with with two component of UMAP
lens5 = mapper.project(X2, projection=umap.UMAP(n_components=2, random_state=rnd), distance_matrix = None, scaler = None)
Umap2_lens = pd.Series(np.concatenate(lens5))  #Cast into Pandas Series


# We create another 1-D lens with with one component of TSNE
lens6 = mapper.project(X2, projection= TSNE(n_components=1, random_state=rnd,perplexity = 1),distance_matrix = None, scaler = None)
Tsne1_lens = pd.Series(np.concatenate(lens6))

# We create another 2-D lens with with two component of TSNE
lens7 = mapper.project(X2, projection= TSNE(n_components=2, random_state=rnd, perplexity = 1,), distance_matrix = None, scaler = None)
Tsne2_lens = pd.Series(np.concatenate(lens7))



#Creo un dizionario con le lenti create
lens = {'IsoForest_lens': [IsoForest_lens],
        'L2Norm_lens':[L2Norm_lens],
        'PCA_lens':[PCA_lens],
        'Umap1_lens': [Umap1_lens],
        'Umap2_lens':[Umap2_lens],
        'Tsne1_lens':[Tsne1_lens],  
        'Tsne2_lens':[Tsne2_lens]}

    
#L'idea è quella di andare a generare le combinazioni in maniera automatica a partire dal dizionario
#Salvo il nome delle lenti da combinare in una variabile
lens_name = list(lens.keys()) 
n_lens = len(lens_name)

#Creo due cicli for in modo da combinare le varie lenti (potrò combinare solo lenti 1D con altre lenti 1D)
for a in lens_name:
    for b in lens_name[lens_name.index(a)+1:]:
        #Crea una chiave con solo a
        dim1 = int(np.array(lens[a]).size)
        dim2 = int(np.array(lens[b]).size)
        #La combinazione tra a e b viene fatta soltanto se entrambi sono di dimensione 1
        if dim1 <= y_class.size & dim2 <= y_class.size:
            key = a[0:a.find('_')] + '_' + b[0:b.find('_')] + '_lens'
            lens[key] = np.c_[lens[a], lens[b]]
            
lens_dict = lens

In [ ]:
lens_accept = [];
print("Eseguo il test di Shapiro per verificare la normalità della lente splittanta rispetto alla classe e successivamente la\
      valuto attravero il test di Mann Whitney la capacità della lente di distinguere e descrivere le due classi.")
for i in lens_name:
    dim = int(np.array(lens[i]).size)
    if dim == y_class.size:
        df2 = pd.DataFrame(np.array(lens[i]).reshape(-1,1), columns = [i])
        df2['class'] = y_class
        lens_class0 = (df2[i].loc[df2["class"] == 0])
        lens_class1 = (df2[i].loc[df2["class"] == 1])
        #Grafico la distribuzione campionaria rispetto a quella Gaussiana rispetto alla classe
        plt.figure(figsize = (6,6))
        sns.distplot(lens_class0, kde = True, color = "c")
        sns.distplot(lens_class1, kde = True, color = "y")
        #plt.plot(x1, scy.norm.pdf(x1, mu1, sigma1))
        plt.legend(['Hist Class_0','Hist Class_1','Dist Campionaria Class_0', 'Dist Campionaria Class_1'])
        plt.title(i + ' vs Class 0 and Class 1')
        plt.savefig('C:/Users/Antonio Montanaro/Desktop/' + i + 'vsClass0_1.png', format = "png", dpi = 1000)
        plt.show()
        
        
        #Eseguo due test statistici (poi  sceglierò soltanto uno)
        #Test di Shapiro molto buono anche per campioni piccoli
        print('1) Eseguo test statistico di Shapiro per verificare la normalità')
        print('-- Il test rifiuta ipotesi H0 che il campione provenga da una distribuzione normale --')
        tShap0, pShap0 = scy.shapiro(lens_class0)
        tShap1, pShap1 = scy.shapiro(lens_class1)
        if pShap0<=0.05 and pShap1<=0.05:
            print('Rifiuto H0. I campioni non provengono da una distribuzione normale')
            print(str(pShap0))
            print(str(pShap1))
        else:
            print('I campioni provengono da una distribuzione normale')    
        
        
            
        print("2) Eseguo il test statistico di MannWhitney per confrontare i due campioni.")
        print('-- Il test rifiuta ipotesi H0 che i due campioni provengano dalla stessa distribuzione --')
        
        t, p = scy.mannwhitneyu(lens_class0,lens_class1)
        print(t)
        if p <= 0.05:
            lens_accept.append(i)
           
            print(i + ' è stata accetta')
            print(str(p*100) + "%")
        else:
            print(i + ' non è stata accetta')
            


In [ ]:
print("Le lenti accettate sono:")
for i in lens_accept:
    print(i)

Dopo aver eseguito una prima classificazione delle lenti tramite test statistico decido di andare ad eseguire una fase più accurata della lente migliore tramite Greedy-Search tra le lenti:
- Tsne2D
- Umap2D

In [ ]:
#Valutazione più accurata della lente Tsne
perp = np.arange(15,55,10) 
learning_rate = np.arange(300,1200,400)
n_iter= np.arange(500,3000, 1000)
entropies_dict_Tsne = {}
for i in perp:
    for j in learning_rate:
        for k in n_iter:
            lensX = mapper.project(X2, projection= TSNE(n_components=2, random_state=17489, perplexity = i, learning_rate=j, n_iter=k),distance_matrix = None, scaler = None)
            Tsne2_lens = pd.DataFrame(lensX)
            scomplex =  mapper.map(Tsne2_lens,
                                    X2,
                                    cover=km.Cover(n_cubes=16, perc_overlap=0.65),  #Modifica rispetto al github causa versione del mapper
                                    clusterer=sklearn.cluster.KMeans(n_clusters=2,random_state=1618033))
            #Funzione che dato il grafo calcola l'entropia
            entropies_dict_Tsne["Tsne2D_perp=",i,"_learnRate=",j,"_nIter=",k] = entropy_count(scomplex) 

Dalla greedy search eseguita nella cella sopra si evince come i parametri che vanno a minimizzare l'entropia sono:
- Perplexity = 45
- Learing Rate = 300
- Number Iteration = 1500

In [ ]:
min_key = min(entropies_dict_Tsne, key=lambda key: entropies_dict_Tsne[key])
print(min_key)

lensX = mapper.project(X2, projection= TSNE(n_components=2, random_state=17489, perplexity = 45, learning_rate=300, n_iter=1500),distance_matrix = None, scaler = None)
Tsne2_lens_final = pd.DataFrame(lensX)

fig = pyplot.figure(figsize=(6, 6))
plt.title("Scatter Plot of Tsne Lens")
plt.suptitle("Tuned Parameters: Perpl = 45 - NIter = 1500 - LearnRate = 300")
plt.scatter(Tsne2_lens_final[0], Tsne2_lens_final[1], c = y_class)
plt.legend(['Class 0','Class 1'])
plt.savefig('Prova11232.png', format='png', dpi=1000)
plt.show()

In [ ]:
#Lente più significativa che analizzo con grid search è: Umap
min_dist = [0.2,0.3,0.5,0.7,0.8]
n_neighbors=[5,10,25,50,120,150,200]
entropies_dict_Umap = {}
for i in n_neighbors:
    for j in min_dist:
        lensX = mapper.project(X2, projection=umap.UMAP(n_components=2, random_state=17489, n_neighbors= i, min_dist=j),distance_matrix = None, scaler = None)
        Umap2_lens = pd.DataFrame(lensX)
        scomplex =  mapper.map(Umap2_lens,
                                    X2,
                                    cover=km.Cover(n_cubes=16, perc_overlap=0.65),  
                                    clusterer=sklearn.cluster.KMeans(n_clusters=2,random_state=1618033))
        #Funzione che dato il grafo calcola l'entropia
        entropies_dict_Umap["Umap2D_MinDist=",j,"_Nneigh=",i] = entropy_count(scomplex) 



Dalla greedy search eseguita nella cella sopra si evince come i parametri che vanno a minimizzare l'entropia sono:
- Min Distance = 0.3
- Number Neighbors = 25

In [ ]:
min_key = min(entropies_dict_Umap, key=lambda key: entropies_dict_Umap[key])
print(min_key)

lensX = mapper.project(X2, projection=umap.UMAP(n_components=2, random_state=rnd, n_neighbors= 25, min_dist=0.3),distance_matrix = None, scaler = None)
Umap2_lens_final = pd.DataFrame(lensX)

fig = pyplot.figure(figsize=(6, 6))
plt.title("Scatter Plot of Umap Lens")
plt.suptitle("Tuned Parameters: Min Distance = 0.3 - N Neighbors = 25")
plt.scatter(Umap2_lens_final[0], Umap2_lens_final[1], c = y_class)
plt.legend(['Class 0','Class 1'])
plt.savefig('Prova11232.png', format='png', dpi=1000)
plt.show()

In [ ]:
min_key = min(entropies_dict_Tsne, key=lambda key: entropies_dict_Tsne[key])
print("Valore di minimo di Entropia per Tsne2D: ",entropies_dict_Tsne[min_key])

min_key = min(entropies_dict_Umap, key=lambda key: entropies_dict_Umap[key])
print("Valore di minimo di Entropia per Umap2D: ",entropies_dict_Umap[min_key])

In [ ]:
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH


pathAbs = str(pathlib.Path().absolute()) 
document = Document()

document.add_heading('Lens Evaluation', 0)

p = document.add_paragraph('Dopo aver generato le diverse lenti, si è passati alla valutazione tramire appositi test-statistici. \n')
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
p.add_run(" L'obiettivo è quello di utilizzare dei test statistici per poter determinare se una lente \
abbia o meno capacità discriminatorie delle classi. I metodi statistici parametrici presumono che \
i dati abbiano una distribuzione nota e specifica, distribuzione gaussiana nel caso del \
t-test ad esempio. Perciò, prima di proseguire nell’esecuzione del t-test sarà necessario \
verificare l’ipotesi di normalità. Ci sono fondamentalmente due strategie per poter valutare \
tale ipotesi, la prima strategia è quella grafica mentre la seconda è quella di applicare un \
test statistico.")

p.add_run("Per quanto riguarda l’approccio grafico, tra i vari strumenti, possiamo andare ad utilizzare \
l’istogramma che ci permette di visualizzare seppur in maniera grossolana, se il campione di \
nostro interesse è aderente o meno alla distribuzione normale.")

p.add_run("Sebbene tramite quest'analisi sia possibile concludere se le distribuzioni siano o meno gaussiane \
per dimostrare in maniera analitica quanto si osserva, è stato eseguito un test statistico di normalità ovvero \
il test di Shapiro-Wilk. Quest'ultimo, è uno dei test più potenti per la verifica della normalità, soprattutto \
per piccoli campioni. La verifica della normalità avviene confrontando due stimatori alternativi della varianza: \
uno stimatore non parametrico basato sulla combinazione lineare ottimale della statistica d'ordine di una variabile \
aleatoria normale al numeratore, e il consueto stimatore parametrico, ossia la varianza campionaria, al denominatore.")

p.add_run("")
 

document.add_heading('Valutazione ipotesi normalità', level=1)
#Creo questo sezione di documento in modo automatico 
for i in lens_name:
    dim = int(np.array(lens[i]).size)
    if dim <= y_class.size:
        d1 = pd.Series(lens[i])
        df2 = pd.DataFrame(np.array(lens[i]).reshape(-1,1), columns = [i])
        df2['class'] = y_class
        lens_class0 = (df2[i].loc[ df2['class'] == 0])
        lens_class1 = (df2[i].loc[ df2['class'] == 1])
        
        document.add_heading(i, level=2)

        document.add_picture(pathAbs + '/LensDistPlot/OriginalData' + i + 'vsClass0_1.png', width=Inches(5))
        
        document.add_paragraph("Test di normalità Shapiro-Wilk")
        #Test di Shapiro-Wilk molto buono anche per campioni piccoli
        tShap0, pShap0 = scy.shapiro(lens_class0)
        tShap1, pShap1 = scy.shapiro(lens_class1)
        if pShap0<=0.05 and pShap1<=0.05:
            document.add_paragraph("Il test rifiuta l'ipotesi nulla H0 ovvero che i due campioni provengano \
da una distribuzione normale con un alpha dello 0.05. \n P-value del " + "{:.18f}".format((pShap0*100)) + " % \
per il campione di classe 0. \n P-value del " + "{:.18f}".format((pShap1*100)) + " % per il campione di classe 1.")
        else:
            document.add_paragraph("Il test non rifiuta l'ipotesi nulla H0 ovvero che i due campioni provengano \
da una distribuzione normale con un alpha dello 0.05. I campioni, dunque, provengono da una distribuzione \
normale.")


document.add_heading('Valutazione Lenti', level=1)
document.add_paragraph("Poichè come visto prima nessun campione è distribuito normalmente, la scelta di applicare \
test parametri non è percorribile. Di conseguenza per poter valutare le capacità della lente di discriminare le \
classi possiamo applicare il test MannWhitney ovvero un test non parametrico per dati non appaiti. \
In particolare, l'ipotesi nulla nel test di Mann-Whitney è quella che i due campioni siano tratti da una popolazione \
singola, e che dunque per questa ragione le loro distribuzioni di probabilità siano uguali. L'ipotesi alternativa \
è che uno dei campioni sia più grande in maniera stocastica. Questo richiede che i due campioni siano statisticamente \
indipendenti e che le osservazioni siano almeno ordinali, o quantitative continue o discrete, perciò questo test si presta \
molto per il nostro scopo, ovvero per il confronto dei due campioni campioni ottenuti dalle diverse lente e \
stratificati rispetto alla classe. Il test lavora rifuitando l'ipotesi nulla (H0: i due campioni provengono dalla \
stessa distribuzione). L'ipotesi alternativa è che uno dei campioni sia più grande in maniera stocastica.")
                       
document.add_paragraph("Seguono i risultati del test per ogni lente: \n")
table = document.add_table(rows=1, cols=3)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Lent'
hdr_cells[1].text = 'P-value'
hdr_cells[2].text = 'Rifiuto H0 [S/N]'
for i in lens_name:
    dim = int(np.array(lens[i]).size)
    if dim <= y_class.size:
        d1 = pd.Series(lens[i])
        df2 = pd.DataFrame(np.array(lens[i]).reshape(-1,1), columns = [i])
        df2['class'] = y_class
        lens_class0 = (df2[i].loc[ df2['class'] == 0])
        lens_class1 = (df2[i].loc[ df2['class'] == 1])
        t, p = scy.mannwhitneyu(lens_class0,lens_class1)
        row_cells = table.add_row().cells
        row_cells[0].text = i
        row_cells[1].text = str(round(p*100,6)) + " %" 
        if p <= 0.05:
            row_cells[2].text = 'S'
        else:
            row_cells[2].text = 'N'

document.add_page_break()

document.save('LensEvaluation1.docx')

In [ ]:
#A questo punto vado ad attuare una greedy search per trovare i parametri di risoluzione che mi minimizzano la lente.
#Questo lo andrò a fare sia per le lente Umap che per la  lente Tsne.
#Lente Tsne - Algoritmo di Cluster Agglomerativo Singol 
#Creo delle mappe che tengono in considerazione soltanto la lente Tsne che ha come parametri min_dist=0.8, n_neighbors=10
#Create a vector of different perc_overlap and n_cubes
perc_over = np.arange(0.3,0.8,0.05)
n_cubes = np.arange(12,22,2)
heartFail_dict = {0: 'Class_0', 1: 'Class_1'}

Tsne2D_Final_Agglomerative = {}
count = 0
for i in n_cubes:
    for j in perc_over:
            scomplex = mapper.map(
                Tsne2_lens_final, 
                X2, 
                clusterer=sklearn.cluster.AgglomerativeClustering(linkage="single", distance_threshold=None),
                cover=km.Cover(n_cubes=i, perc_overlap=j) #j e k sono rispettivamente gli elementi dei vettori 'n_cubes' e 'perc_overlap'
            )
            
            title =  "Lens = Tsne2D - Clustering = Agglomerative - Entropia = " + str(entropy_count(scomplex))
            Tsne2D_Final_Agglomerative["Lens = Tsne2D - Clustering = Agglomerative - N_cubes =" + str(i) +"- %_overlap =" + str(j)] = entropy_count(scomplex)
            count = count + 1
            fw_graph = do_graph(scomplex,title,y_class, pl_brewer)
            fw_graph.write_image("ImageFinals\ImmaginiTsne\ClusterAgglomerative\Image"+ str(count)+".jpeg" ,width=800, height=800)
            do_hist_counter(scomplex,"LensTsne2D - Agglormerative - N_cub=" + str(i) +"- %_overlap =" + str(j) + "\n" + "Distribuzione della dimensione dei nodi", "ImageFinals\ImmaginiTsne\ClusterAgglomerative\HistImage"+ str(count)+".jpeg")
    
            color_class = do_color_class(scomplex,y_class,heartFail_dict,pl_brewer)
            #Utilizzo una funzione del kmapper per trasformarel'output del mapper in una rete NetworkX
            G = km.adapter.to_nx(scomplex)
            node_color, internal, internal_color, external = community_searching(G)
            path = "ImageFinalsNetworkX\ImmaginiTsne\ClusterAgglomerative\Image"+ str(count)+".jpeg"
            do_graph_NetworkX(G,color_class, path,node_color, internal, internal_color)


Tsne2D_Final_DBSCAN = {}
count = 0
for i in n_cubes:
    for j in perc_over:
            scomplex = mapper.map(
                Tsne2_lens_final, 
                X2, 
                clusterer=sklearn.cluster.DBSCAN(),
                cover=km.Cover(n_cubes=i, perc_overlap=j)  #j e k sono rispettivamente gli elementi dei vettori 'n_cubes' e 'perc_overlap'
            )
            
            title =  "Lens = Tsne2D - Clustering = DBSCAN - Entropia = " + str(entropy_count(scomplex))
            Tsne2D_Final_DBSCAN["Lens = Tsne2D - Clustering = DBSCAN - N_cubes =" + str(i)+ "- %_overlap =" + str(j)] = entropy_count(scomplex)
            count = count + 1
            fw_graph = do_graph(scomplex,title,y_class, pl_brewer)
            fw_graph.write_image("ImageFinals\ImmaginiTsne\DBSCAN\Image"+ str(count)+".jpeg" ,width=800, height=800)
            do_hist_counter(scomplex,"LensTsne2D - DBSCAN - N_cub=" + str(i) +"- %_overlap =" + str(j) + "\n" + "Distribuzione della dimensione dei nodi", "ImageFinals\ImmaginiTsne\DBSCAN\HistImage"+ str(count)+".jpeg")
            
            color_class = do_color_class(scomplex,y_class,heartFail_dict,pl_brewer)
            #Utilizzo una funzione del kmapper per trasformarel'output del mapper in una rete NetworkX
            G = km.adapter.to_nx(scomplex)
            node_color, internal, internal_color, external = community_searching(G)
            path = "ImageFinalsNetworkX\ImmaginiTsne\DBSCAN\Image"+ str(count)+".jpeg"
            do_graph_NetworkX(G,color_class, path,node_color, internal, internal_color)

Tsne2D_Final_KMeans = {}
count = 0
for i in n_cubes:
    for j in perc_over:
            scomplex = mapper.map(
                Tsne2_lens_final, 
                X2, 
                clusterer=sklearn.cluster.KMeans(n_clusters=2, random_state=1618033),
                cover=km.Cover(n_cubes=i, perc_overlap=j)  #j e k sono rispettivamente gli elementi dei vettori 'n_cubes' e 'perc_overlap'
            )

            title =  "Lens = Tsne2D - Clustering = KMeans - Entropia = " + str(entropy_count(scomplex))
            Tsne2D_Final_KMeans["Lens = Tsne2D - Clustering = KMeans - N_cubes =" + str(i) +"- %_overlap =" + str(j)] = entropy_count(scomplex)
            count = count + 1
            fw_graph = do_graph(scomplex,title,y_class, pl_brewer)
            fw_graph.write_image("ImageFinals\ImmaginiTsne\ClusterKMeans\Image"+ str(count)+".jpeg" ,width=800, height=800)
            do_hist_counter(scomplex,"LensTsne2D - KMeans - N_cub=" + str(i) +"- %_overlap =" + str(j) + "\n" + "Distribuzione della dimensione dei nodi", "ImageFinals\ImmaginiTsne\ClusterKMeans\HistImage"+ str(count)+".jpeg")
            
            color_class = do_color_class(scomplex,y_class,heartFail_dict,pl_brewer)
            #Utilizzo una funzione del kmapper per trasformarel'output del mapper in una rete NetworkX
            G = km.adapter.to_nx(scomplex)
            node_color, internal, internal_color, external = community_searching(G)
            path = "ImageFinalsNetworkX\ImmaginiTsne\ClusterKMeans\Image"+ str(count)+".jpeg"
            do_graph_NetworkX(G,color_class, path, node_color, internal, internal_color)


In [ ]:
#Lente Umap - Algoritmo di Cluster Agglomerativo Singol 
#Creo delle mappe che tengono in considerazione soltanto la lente Tsne che ha come parametri min_dist=0.8, n_neighbors=10
#Create a vector of different perc_overlap and n_cubes
perc_over = np.arange(0.3,0.8,0.05)
n_cubes = np.arange(12,22,2)

Umap2D_Final_Agglomerative = {}

count = 0
for i in n_cubes:
    for j in perc_over:
            scomplex = mapper.map(
                Umap2_lens_final, 
                X2, 
                clusterer=sklearn.cluster.AgglomerativeClustering(linkage="single", distance_threshold=None),
                cover=km.Cover(n_cubes=i, perc_overlap=j) #j e k sono rispettivamente gli elementi dei vettori 'n_cubes' e 'perc_overlap'
                
            )
            
            title =  "Lens = Umap2D - Clustering = Agglomerative - Entropia = " + str(entropy_count(scomplex))
            Umap2D_Final_Agglomerative["Lens = Umap2D - Clustering = Agglomerative - N_cubes =" + str(i)+ "- %_overlap =" + str(j)] = entropy_count(scomplex)
            count = count + 1
            fw_graph = do_graph(scomplex,title,y_class, pl_brewer)
            fw_graph.write_image("ImageFinals\ImmaginiUmap\ClusterAgglomerative\Image"+ str(count)+".jpeg" ,width=800, height=800)
            do_hist_counter(scomplex,"LensUmap2D - Agglormerative - N_cub=" + str(i) +"- %_overlap =" + str(j) + "\n" + "Distribuzione della dimensione dei nodi", "ImageFinals\ImmaginiUmap\ClusterAgglomerative\HistImage"+ str(count)+".jpeg")
            
            color_class = do_color_class(scomplex,y_class,heartFail_dict,pl_brewer)
            #Utilizzo una funzione del kmapper per trasformarel'output del mapper in una rete NetworkX
            G = km.adapter.to_nx(scomplex)
            node_color, internal, internal_color, external = community_searching(G)
            path = "ImageFinalsNetworkX\ImmaginiUmap\ClusterAgglomerative\Image"+ str(count)+".jpeg"
            do_graph_NetworkX(G,color_class, path, node_color, internal, internal_color)
            

Umap2D_Final_DBSCAN = {}
count = 0
for i in n_cubes:
    for j in perc_over:
            scomplex = mapper.map(
                Umap2_lens_final, 
                X2, 
                clusterer=sklearn.cluster.DBSCAN(),
                cover=km.Cover(n_cubes=i, perc_overlap=j)  #j e k sono rispettivamente gli elementi dei vettori 'n_cubes' e 'perc_overlap'
            )
            
            title =  "Lens = Umap2D - Clustering = DBSCAN - Entropia = " + str(entropy_count(scomplex))
            Umap2D_Final_DBSCAN["Lens = Umap2D - Clustering = DBSCAN - N_cubes =" + str(i) +"- %_overlap =" + str(j)] = entropy_count(scomplex)
            count = count + 1
            fw_graph = do_graph(scomplex,title,y_class, pl_brewer)
            fw_graph.write_image("ImageFinals\ImmaginiUmap\DBSCAN\Image"+ str(count)+".jpeg" ,width=800, height=800)
            do_hist_counter(scomplex,"LensUmap2D - DBSCAN - N_cub=" + str(i) +"- %_overlap =" + str(j) + "\n" + "Distribuzione della dimensione dei nodi", "ImageFinals\ImmaginiUmap\DBSCAN\HistImage"+ str(count)+".jpeg")
            
            color_class = do_color_class(scomplex,y_class,heartFail_dict,pl_brewer)
            #Utilizzo una funzione del kmapper per trasformarel'output del mapper in una rete NetworkX
            G = km.adapter.to_nx(scomplex)
            node_color, internal, internal_color, external = community_searching(G)
            path = "ImageFinalsNetworkX\ImmaginiUmap\DBSCAN\Image"+ str(count)+".jpeg"
            do_graph_NetworkX(G,color_class, path, node_color, internal, internal_color)
            
Umap2D_Final_KMeans = {}
count = 0
for i in n_cubes:
    for j in perc_over:
            scomplex = mapper.map(
                Umap2_lens_final, 
                X2, 
                clusterer=sklearn.cluster.KMeans(n_clusters=2, random_state=1618033),
                cover=km.Cover(n_cubes=i, perc_overlap=j)  #j e k sono rispettivamente gli elementi dei vettori 'n_cubes' e 'perc_overlap'
            )

            title =  "Lens = Umap2D - Clustering = KMeans - Entropia = " + str(entropy_count(scomplex))
            Umap2D_Final_KMeans["Lens = Umap2D - Clustering = KMeans - N_cubes =" + str(i)+ "- %_overlap =" + str(j)] = entropy_count(scomplex)
            count = count + 1
            fw_graph = do_graph(scomplex,title,y_class, pl_brewer)
            fw_graph.write_image("ImageFinals\ImmaginiUmap\ClusterKMeans\Image"+ str(count)+".jpeg" ,width=800, height=800)
            do_hist_counter(scomplex,"LensUmap2D - KMeans - N_cub=" + str(i) +"- %_overlap =" + str(j) + "\n" + "Distribuzione della dimensione dei nodi", "ImageFinals\ImmaginiUmap\ClusterKMeans\HistImage"+ str(count)+".jpeg")
            
            color_class = do_color_class(scomplex,y_class,heartFail_dict,pl_brewer)
            #Utilizzo una funzione del kmapper per trasformarel'output del mapper in una rete NetworkX
            G = km.adapter.to_nx(scomplex)
            node_color, internal, internal_color,external = community_searching(G)
            path = "ImageFinalsNetworkX\ImmaginiUmap\ClusterKMeans\Image"+ str(count)+".jpeg"
            do_graph_NetworkX(G,color_class, path, node_color, internal, internal_color)

In [ ]:
import pickle
def save_dict(dictionary, path):
    a_file = open(path, "wb")
    pickle.dump(dictionary, a_file)
    a_file.close()
    
print("Tsne Lens")
min_key = min(Tsne2D_Final_Agglomerative, key=lambda key: Tsne2D_Final_Agglomerative[key])
print("Valore di minimo di Entropia per Tsne2D: ",Tsne2D_Final_Agglomerative[min_key],"\n", min_key)
save_dict(Tsne2D_Final_Agglomerative,"C:/Users/Antonio Montanaro/Desktop/Tesi Magistrale/File Memory/Dictionary/Tsne2D_Final_Agglomerative.pkl")

min_key = min(Tsne2D_Final_KMeans, key=lambda key: Tsne2D_Final_KMeans[key])
print("Valore di minimo di Entropia per Tsne2D: ",Tsne2D_Final_KMeans[min_key],"\n", min_key)
save_dict(Tsne2D_Final_KMeans,"C:/Users/Antonio Montanaro/Desktop/Tesi Magistrale/File Memory/Dictionary/Tsne2D_Final_KMeans.pkl")

min_key = min(Tsne2D_Final_DBSCAN, key=lambda key: Tsne2D_Final_DBSCAN[key])
print("Valore di minimo di Entropia per Tsne2D: ",Tsne2D_Final_DBSCAN[min_key],"\n", min_key)
save_dict(Tsne2D_Final_DBSCAN,"C:/Users/Antonio Montanaro/Desktop/Tesi Magistrale/File Memory/Dictionary/Tsne2D_Final_DBSCAN.pkl")

print("\n")

print("Umap Lens")
min_key = min(Umap2D_Final_Agglomerative, key=lambda key: Umap2D_Final_Agglomerative[key])
print("Valore di minimo di Entropia per Umap2D: ",Umap2D_Final_Agglomerative[min_key],"\n", min_key)
save_dict(Umap2D_Final_Agglomerative,"C:/Users/Antonio Montanaro/Desktop/Tesi Magistrale/File Memory/Dictionary/Umap2D_Final_Agglomerative.pkl")


min_key = min(Umap2D_Final_KMeans, key=lambda key: Umap2D_Final_KMeans[key])
print("Valore di minimo di Entropia per Umap2D: ",Umap2D_Final_KMeans[min_key],"\n", min_key)
save_dict(Umap2D_Final_KMeans,"C:/Users/Antonio Montanaro/Desktop/Tesi Magistrale/File Memory/Dictionary/Umap2D_Final_KMeans.pkl")


min_key = min(Umap2D_Final_DBSCAN, key=lambda key: Umap2D_Final_DBSCAN[key])
print("Valore di minimo di Entropia per Umap2D: ",Umap2D_Final_DBSCAN[min_key],"\n", min_key)
save_dict(Umap2D_Final_DBSCAN,"C:/Users/Antonio Montanaro/Desktop/Tesi Magistrale/File Memory/Dictionary/Umap2D_Final_DBSCAN.pkl")

Prendiamo in analisi Tsne2D con:
- % Overlap = 0.55
- N cubes = 18
- Algoritmo CLustering = KMeans

In [ ]:
#Eseguo i BoxPlot per poter confrontare le entropie dei diversi metodi di clustering con successivo test statistico
dataDictionaryLens = pd.DataFrame()
dataDictionaryLens["Tsne2D_Agglomerative"] = Tsne2D_Final_Agglomerative.values()
dataDictionaryLens["Tsne2D_KMeans"] = Tsne2D_Final_KMeans.values()
dataDictionaryLens["Tsne2D_DBSCAN"] = Tsne2D_Final_DBSCAN.values()
dataDictionaryLens["Umap2D_Agglomerative"] = Umap2D_Final_Agglomerative.values()
dataDictionaryLens["Umap2D_KMeans"] = Umap2D_Final_KMeans.values()
dataDictionaryLens["Umap2D_DBSCAN"] =  prova


In [ ]:
fig = plt.figure(figsize=(12,6))
sns.boxplot(data=dataDictionaryLens)
plt.title("Boxplot Entropie diversi algoritmi di clustering")

In [ ]:
#Verico la normalità dei vari vettori di entropia
for i in dataDictionaryLens.columns:
    fstat, pval =scy.shapiro(dataDictionaryLens[i])
    if pval <= 0.05:
        print(i + " è distribuito normalemente")
    else:
        print(i + "non è distribuito normalemente")
#Poichè tutti i vettori sono distribuiti normalmente posso applicare il test anova che va a verificare se le medie dei
#vettori sono uguali, se dovessi rifiutare l'ipotesi nulla potrei dire che le medie sono diverse ma non saprei dire quale
#è significativamente diversa, occorrera quindi eseguire un nuovo test.
print("")
fvalue, pvalue =scy.f_oneway(dataDictionaryLens["Tsne2D_Agglomerative"], dataDictionaryLens["Tsne2D_KMeans"], dataDictionaryLens["Tsne2D_DBSCAN"], dataDictionaryLens["Umap2D_Agglomerative"], dataDictionaryLens["Umap2D_KMeans"], dataDictionaryLens["Umap2D_DBSCAN"])
if pvalue<0.05:
    print("Le medie dei vettori sono significativamente diverse! Occorre eseguire un test di Tukey!")
else:
    print("Le medie non sono significativemente diverse!")

In [ ]:
#Preparo il dataframe per il test statistico
lista = dataDictionaryLens["Tsne2D_Agglomerative"].tolist() + dataDictionaryLens["Tsne2D_KMeans"].tolist()+ dataDictionaryLens["Tsne2D_DBSCAN"].tolist() + dataDictionaryLens["Umap2D_Agglomerative"].tolist() + dataDictionaryLens["Umap2D_KMeans"].tolist()+ dataDictionaryLens["Umap2D_DBSCAN"].tolist() 
df = pd.DataFrame({'score': lista,
                   'group': np.repeat(['Tsne_Aggl', 'Tsne_KMeans','Tsne_DBSCAN','Umap_Aggl','Umap_KMeans','Umap_DBSCAN'], repeats=len(dataDictionaryLens))}) 

tukey = pairwise_tukeyhsd(endog=df['score'],groups=df['group'],alpha=0.05)
print(tukey)
print("I risultati devono essere interpretati facendo riferimento al meandiff, dove questo è piccolo indica una bassa\
 differenza")


Dai risultati sopra risulta evidente che il clustering con DBSCAN sia quello più scadente. Quindi si prenderà in considerazione il KMeans

In [ ]:
del scomplex
lensX = mapper.project(X2, projection= TSNE(n_components=2, random_state=17489, perplexity = 45, learning_rate=300, n_iter=1500),distance_matrix = None, scaler = None)
Tsne2_lens_final = pd.DataFrame(lensX)

scomplex = mapper.map(
                Tsne2_lens_final, 
                X2, 
                clusterer=sklearn.cluster.KMeans(n_clusters=2, random_state=1618033),
                cover=km.Cover(n_cubes=18, perc_overlap=0.55)  #j e k sono rispettivamente gli elementi dei vettori 'n_cubes' e 'perc_overlap'
)
    
title =  "Lens = Tsne2D - Clustering = KMeans - Entropia = " + str(entropy_count(scomplex))


In [ ]:
kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex,color_function_name='Distance to x-min')

In [ ]:
eta = dataImputed["Età"]          #Prendo il vettore eta non scalato                               
Age_scaled = datasetImputed["Età"]#Prendo il vettore età scalato

anniFumo = dataImputed["Anni da fumatore"]           #Prendo il vettore eta non scalato                               
anniFumo_scaled = datasetImputed["Anni da fumatore"] #Prendo il vettore età scalato

peso = dataImputed["Peso "]           #Prendo il vettore eta non scalato                               
peso_scaled = datasetImputed["Peso "] #Prendo il vettore età scalato

Gamma = dataImputed["Gamma-GT "]         #Prendo il vettore eta non scalato                               
Gamma_scaled = datasetImputed["Gamma-GT "]   #Prendo il vettore età scalato

HDLcolesterolo = dataImputed["Colesterolo HDL "]             #Prendo il vettore eta non scalato                               
HDLcolesterolo_scaled = datasetImputed["Colesterolo HDL "]   #Prendo il vettore età scalato

presSistolica = dataImputed["Pressione arteriosa sistolica "]           #Prendo il vettore eta non scalato                               
presSistolica_scaled = datasetImputed["Pressione arteriosa sistolica "]   #Prendo il vettore età scalato

LDLcolesterolo = dataImputed["Colesterolo LDL "]           #Prendo il vettore eta non scalato                               
LDLcolesterolo_scaled = datasetImputed["Colesterolo LDL "]   #Prendo il vettore età scalato


risk_score = pd.read_csv("C:/Users/Antonio Montanaro/Desktop/Tesi Magistrale/Risk_score.csv")
fw_rischio = do_enrinchment(scomplex,pl_brewer, risk_score["Risk_score"],risk_score["Risk_score"], fwn)
VBox([HBox([fwn,fw_rischio])])

In [ ]:
var_cat = datasetImputed["Il paziente soffre di diabete mellito?_1"]
fw_diabetic = do_enrinchment_cat(scomplex,pl_brewer, var_cat, fwn, "of the diabetic disease", {0:"Cat_0", 1:"Cat_1"})
fw_malattiaCard = do_enrinchment_cat(scomplex,pl_brewer, datasetImputed['Il paziente ha mai avuto sintomi di malattia cardiaca?_1'], fwn, "of cardiac disease", {0:"Cat_0", 1:"Cat_1"})
fw_sex = do_enrinchment_cat(scomplex,pl_brewer, datasetImputed["Sesso"], fwn, "of sex", {0:"Cat_0", 1:"Cat_1"})


VBox([HBox([fwn, fw_diabetic]), HBox([fwn, fw_malattiaCard]), HBox([fwn, fw_sex])])

In [ ]:
a =np.random.seed(42)
g_pos = nx.spring_layout
G = km.adapter.to_nx(scomplex)
plt.rcParams.update({'figure.figsize': (15, 10)})

# Grafico la stessa rete con i pacchetti NetworkX e igraph
fig, (ax0,ax1) = plt.subplots(nrows=1, ncols=2, figsize=(14, 8))
ax0.set_title("Plot with NetworkX - Class Proportion")
nx.colormaps = pl_brewer
nx.draw_kamada_kawai(G, node_size=50, ax=ax0, node_color=do_color_class(scomplex,y_class,heartFail_dict,pl_brewer), cmap = 'coolwarm')
#la color map coolwarm va dal blu (0) al rosso (1) quindi nei nodi blu vi è una prevalenza di soggetti sani

G = km.adapter.to_nx(scomplex)
node_color, internal, internal_color, external = community_searching(G)

ax1.set_title("Plot with NetworkX - Communities")
nx.draw_kamada_kawai(G, node_size=50, ax=ax1, node_color= node_color, edgelist=internal, edge_color = internal_color)


# Grafico la stessa rete con i pacchetti NetworkX e igraph
fig, (ax0,ax1) = plt.subplots(nrows=1, ncols=2, figsize=(14, 8))

ax0.set_title("Plot with NetworkX - Class Proportion")
nx.colormaps = pl_brewer
nx.draw_kamada_kawai(G, node_size=50, ax=ax0, node_color=do_color_class(scomplex,y_class,heartFail_dict,pl_brewer), cmap = 'coolwarm')
#la color map coolwarm va dal blu (0) al rosso (1) quindi nei nodi blu vi è una prevalenza di soggetti sani

risk_score = pd.read_csv("C:/Users/Antonio Montanaro/Desktop/File Memory/Tesi Magistrale/Risk_score.csv")
color = do_color(scomplex, risk_score["Risk_score"], risk_score["Risk_score"], heartFail_dict,pl_brewer)
color_rgb = []
for i in color:
    if i<0.03:
        color_rgb.append("#1d3557")
    if i>0.30:
        color_rgb.append("#e63946")
    if i >=0.03 and i<0.3:
        color_rgb.append("#a8dadc")
ax1.set_title("Plot with NetworkX - Risk Score")
nx.draw_kamada_kawai(G, node_size=50, ax=ax1, node_color=color_rgb)

In [ ]:
del risk_score["Unnamed: 0"]
#Implemento il chord diagram
paz_information = risk_score.copy()
paz_information["Id_paz"] = range(0, len(paz_information))

#Aggiungo una colonna in cui vi è la classe di rischio
for i in paz_information.iterrows():
    if i[1][0]<=0.03:
        paz_information.at[i[0],'Class_Risk'] = "Basso"
        paz_information.at[i[0],'Color_Risk'] = "#1d3557"
        
    if i[1][0]>0.03 and i[1][0]<=0.3:
        paz_information.at[i[0],'Class_Risk'] = "Medio"
        paz_information.at[i[0],'Color_Risk'] = "#a8dadc"
    
    if i[1][0]>0.3:
        paz_information.at[i[0],'Class_Risk'] = "Alto"
        paz_information.at[i[0],'Color_Risk'] = "#e63946"

#In questo modo so ogni nodo a quale communities appartiene
communities = ["Comm_"+ str(x) for x in range(1, len(set(node_color))+1)]
color_communities = list(set(node_color))

node_communities = []
color_node_comm = []
for i, k in enumerate(set(node_color)):
    for j in node_color:
        if j == k:
            node_communities.append(communities[i])
            color_node_comm.append(j)
    
#L'obiettivo è associare ad ogni paziente la communities e poi il rischio 
#Creo un dataframe con il paziente come chiare associato alla communities
for i, node in enumerate(G.nodes()):
    for j in paz_information.iterrows():
        if j[0] in scomplex["nodes"].get(node):
            paz_information.at[j[0], 'Communities'] = node_communities[i]
            paz_information.at[j[0], 'Color_Comm'] = str(color_node_comm[i])

In [ ]:
find_file("../File Memory/dataframe_Communities.csv", paz_information ,0)

In [ ]:
#Aggiungo informazione al dataframe dei pazienti
paz_information["Età"]  = eta
paz_information["Peso"] = peso
paz_information["Anni Fumo"] = anniFumo
paz_information["HDL Colesterolo"] = HDLcolesterolo
paz_information["LDL Colesterolo"] = LDLcolesterolo
paz_information["Pressione Sistolica"]  = presSistolica 
paz_information["Sesso"] = datasetImputed["Sesso"]
paz_information["BMI"] = datasetImputed["BMI"]
paz_information['Il paziente soffre di diabete mellito?'] = datasetImputed['Il paziente soffre di diabete mellito?_1']
paz_information['Il paziente soffre di ipercolesterolemia?'] = datasetImputed['Il paziente soffre di ipercolesterolemia?_1']
paz_information['Il paziente soffre di arteriopatia periferica?'] = datasetImputed['Il paziente soffre di arteriopatia periferica?_1']
paz_information['Il paziente soffre di anemia?'] = datasetImputed['Il paziente soffre di anemia?_1']
paz_information['Il paziente soffre di insufficienza renale cronica?'] = datasetImputed['Il paziente soffre di insufficienza renale cronica?_1']
paz_information['Il paziente soffre di sindrome trombofilica?'] = datasetImputed['Il paziente soffre di sindrome trombofilica?_1']
paz_information['Il paziente ha mai avuto sintomi di malattia cardiaca?'] = datasetImputed['Il paziente ha mai avuto sintomi di malattia cardiaca?_1']

In [ ]:
colors_dict = {"Basso" : '#1d3557',
               "Medio" : '#a8dadc',
               "Alto" : '#e63946', 
               "Comm_1": color_communities[0],
               "Comm_2": color_communities[1],
               "Comm_3": color_communities[2],
               "Comm_4": color_communities[3],
               "Comm_5": color_communities[4],
               "Comm_6": color_communities[5]}

sankey(left = paz_information["Class_Risk"], right = paz_information["Communities"], colorDict= colors_dict, aspect=20, fontsize=12)

In [ ]:
sankey(left = paz_information["Communities"], right = paz_information["Class_Risk"], colorDict= colors_dict, aspect=20, fontsize=12)

In [ ]:
#Grafico, all'interno delle varie communities gli istogrammi o box plot delle variabili considerata più informative:
var = ["Peso", "Età", "Anni Fumo", "HDL Colesterolo", "LDL Colesterolo"]
color_communities = list(set(node_color))
figure = plt.figure(figsize=(27, 24))
k = 1
for i in var:
    for j, comm in enumerate(communities):
        ax = plt.subplot(len(set(paz_information["Communities"])), len(var) + 1, k)
        x = paz_information.loc[paz_information["Communities"] == comm]
        ax.hist(x[i], color = color_communities[j])
        ax.set_title(comm + "_Var=" + i)
        k = k +1


In [ ]:
dict_communities = {}
for i in communities:
    dict_communities.update({i: paz_information.loc[paz_information["Communities"] == i]})

In [ ]:
comm_selected = ["Comm_4","Comm_5","Comm_6"]
col= ['Risk_score','Età', 'Peso', 'Anni Fumo', 'HDL Colesterolo',
       'LDL Colesterolo', 'Pressione Sistolica', 'Sesso', 'BMI',
       'Il paziente soffre di diabete mellito?',
       'Il paziente soffre di ipercolesterolemia?',
       'Il paziente soffre di arteriopatia periferica?',
       'Il paziente soffre di anemia?',
       'Il paziente soffre di insufficienza renale cronica?',
       'Il paziente soffre di sindrome trombofilica?',
       'Il paziente ha mai avuto sintomi di malattia cardiaca?']
cat = ['Sesso',
       'Il paziente soffre di diabete mellito?',
       'Il paziente soffre di ipercolesterolemia?',
       'Il paziente soffre di arteriopatia periferica?',
       'Il paziente soffre di anemia?',
       'Il paziente soffre di insufficienza renale cronica?',
       'Il paziente soffre di sindrome trombofilica?',
       'Il paziente ha mai avuto sintomi di malattia cardiaca?']
groupby = 'Class_Risk'


k = 1
for i in comm_selected:
    x = dict_communities[i]
    print(TableOne(data = x, columns=col, categorical=cat,groupby=groupby))


figure = plt.figure(figsize=(5, 5)) 
plt.hist(dict_communities["Comm_4"]["Risk_score"], alpha=0.5, label='Comm 4')
plt.hist(dict_communities["Comm_5"]["Risk_score"], alpha=0.5, label='Comm 5')
plt.hist(dict_communities["Comm_6"]["Risk_score"], alpha=0.5, label='Comm 6')
plt.legend(loc='upper right')
plt.show()

In [ ]:
tab = TableOne(data = x, columns=col, categorical=cat,groupby=groupby)
tab.to_latex("prova.tex")